## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dabakala,CI,8.37,-4.43,70.95,95,90,6.58,overcast clouds
1,1,Mataura,NZ,-46.19,168.86,52.25,64,100,11.07,overcast clouds
2,2,Vaini,TO,-21.20,-175.20,78.80,78,75,9.17,broken clouds
3,3,Kaihua,CN,23.37,104.28,78.12,67,82,7.36,broken clouds
4,4,Bredasdorp,ZA,-34.53,20.04,46.40,87,81,9.17,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.20,-175.20,78.80,78,75,9.17,broken clouds
3,3,Kaihua,CN,23.37,104.28,78.12,67,82,7.36,broken clouds
6,6,Gladstone,AU,-23.85,151.25,75.20,41,9,20.80,clear sky
7,7,Bengkulu,ID,-3.80,102.27,85.37,63,32,7.85,scattered clouds
16,16,Isangel,VU,-19.55,169.27,78.80,88,75,9.17,broken clouds
20,20,Ponta Do Sol,PT,32.67,-17.10,77.00,69,40,10.29,light intensity drizzle rain
26,26,Kapaa,US,22.08,-159.32,82.40,65,20,13.87,few clouds
30,30,Caucaia,BR,-3.74,-38.65,79.00,78,20,13.87,few clouds
31,31,Butaritari,KI,3.07,172.79,82.42,74,68,12.88,broken clouds
32,32,Lazaro Cardenas,MX,17.96,-102.20,78.96,83,83,4.27,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                222
City                   222
Country                222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,78.80,broken clouds,-21.20,-175.20,
3,Kaihua,CN,78.12,broken clouds,23.37,104.28,
6,Gladstone,AU,75.20,clear sky,-23.85,151.25,
7,Bengkulu,ID,85.37,scattered clouds,-3.80,102.27,
16,Isangel,VU,78.80,broken clouds,-19.55,169.27,
20,Ponta Do Sol,PT,77.00,light intensity drizzle rain,32.67,-17.10,
26,Kapaa,US,82.40,few clouds,22.08,-159.32,
30,Caucaia,BR,79.00,few clouds,-3.74,-38.65,
31,Butaritari,KI,82.42,broken clouds,3.07,172.79,
32,Lazaro Cardenas,MX,78.96,broken clouds,17.96,-102.20,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]    
    except (IndexError):         
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   222
Country                222
Max Temp               222
Current Description    222
Lat                    222
Lng                    222
Hotel Name             222
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search\WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))